嘗試取得指定債券的歷史資料並繪圖

In [ ]:
import requests
import pandas as pd
from datetime import datetime

# URL 從哪裡取得資料
url = "https://markets.businessinsider.com/Ajax/Chart_GetChartData?instrumentType=Bond&tkData=1,46441575,1330,333&from=19700201&to=20250220"

# 發送請求取得資料
response = requests.get(url)

# 檢查響應狀態碼，確保請求成功
if response.status_code == 200:
    # 解析 JSON 數據
    data = response.json()

    # 轉換為 DataFrame
    df = pd.DataFrame(data)

    # 將日期從字串轉換為 datetime 對象
    df['Date'] = pd.to_datetime(df['Date'])

    # 儲存為 Excel 文件
    excel_file = 'data/MI_歷史數據.xlsx'
    df.to_excel(excel_file, index=False)

    print(f"數據已儲存到 {excel_file}")
else:
    print("Failed to retrieve data:", response.status_code)


取得多支債券數據

In [ ]:
import requests
import pandas as pd
from datetime import datetime

# ISIN 對應的 tkData
isin_to_tkdata = {
    'US02209SBF92': ('高特力 2039 5.95', '1,46441569,1330,333'),
    'US037833BX70': ('蘋果 2046 4.65', '1,31618402,1330,333'),
    'US02209SBE28': ('高特力 2039 5.8', '1,46441575,1330,333'),
    'US716973AH54': ('輝瑞 2053 5.3', '1,127132136,1330,333'),
    'US842434DA71': ('南加州天然氣 2054 5.6', ''),  # 尚未查到
    'US872898AJ06': ('台積電 2052 4.5', '1,118393079,16,333'),
    'USF2893TAE67': ('法國電力 2040 5.6', '1,10955366,1330,333'),
    'US05526DBV64': ('英美菸草 2052 4.65', '1,117582253,1330,333'),
    'US717081ED10': ('輝瑞 2046 4.125', ''),  # 尚未查到
    'US716973AG71': ('輝瑞 2053 5.3', '1,127131476,1330,333')
}

# 查詢時間範圍
start_date = "19700201"
end_date = "20250220"

# 儲存所有數據的 DataFrame 字典
all_data = {}

# 遍歷每個 ISIN，查詢價格數據
for isin, (bond_name, tk_data) in isin_to_tkdata.items():
    print(f"\n🔍 正在查詢 {bond_name} ({isin}) ...")

    if not tk_data:
        print(f"⚠️ 無法取得 {bond_name} ({isin}) 的 tkData，跳過")
        # 若無 tkData，直接跳過
        continue

    # 設定 API URL
    url = f"https://markets.businessinsider.com/Ajax/Chart_GetChartData?instrumentType=Bond&tkData={tk_data}&from={start_date}&to={end_date}"
    
    # 發送請求
    response = requests.get(url)

    if response.status_code == 200:
        try:
            data = response.json()
            # 若 API 回傳空數據，則跳過
            if not data:
                print(f"⚠️ {bond_name} ({isin}) 無數據，跳過")
                continue

            # 轉換為 DataFrame
            df = pd.DataFrame(data)
            # 日期格式 yyyy/mm/dd
            df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y/%m/%d')

            # 只保留 Close 欄位
            df = df[['Date', 'Close']]

            # 加入 ISIN 和 Bond Name
            df.insert(1, 'ISIN', isin)
            df.insert(2, 'Bond Name', bond_name)

            # 加入至數據字典
            all_data[bond_name] = df
            print(
                f"✅ {bond_name} ({isin}) 數據取得成功，"
                f"共 {len(df)} 筆"
            )

        except Exception as e:
            print(
                f"❌ 解析 {bond_name} ({isin}) "
                f"JSON 失敗: {e}"
            )
    else:
        print(
            f"❌ {bond_name} ({isin}) 查詢失敗，"
            f"HTTP 狀態碼: {response.status_code}"
        )

# 儲存至 Excel，每支債券分開存放在不同的 Sheet
if all_data:
    excel_file = 'data/MI_歷史數據_全.xlsx'
    with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:
        for sheet_name, df in all_data.items():
            # Excel Sheet 名稱最多 31 字元
            sheet_name = sheet_name[:31]
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    print(
        f"\n📊 所有數據已儲存至 {excel_file}，"
        "每支債券單獨存放在不同的 Sheet"
    )
else:
    print("\n⚠️ 無有效數據，未儲存 Excel 檔案")



🔍 正在查詢 高特力 2039 5.95 (US02209SBF92) ...
✅ 高特力 2039 5.95 (US02209SBF92) 數據取得成功，共 1532 筆

🔍 正在查詢 蘋果 2046 4.65 (US037833BX70) ...
✅ 蘋果 2046 4.65 (US037833BX70) 數據取得成功，共 2285 筆

🔍 正在查詢 高特力 2039 5.8 (US02209SBE28) ...
✅ 高特力 2039 5.8 (US02209SBE28) 數據取得成功，共 1532 筆

🔍 正在查詢 輝瑞 2053 5.3 (US716973AH54) ...
✅ 輝瑞 2053 5.3 (US716973AH54) 數據取得成功，共 450 筆

🔍 正在查詢 南加州天然氣 2054 5.6 (US842434DA71) ...
⚠️ 無法取得 南加州天然氣 2054 5.6 (US842434DA71) 的 tkData，跳過

🔍 正在查詢 台積電 2052 4.5 (US872898AJ06) ...
✅ 台積電 2052 4.5 (US872898AJ06) 數據取得成功，共 720 筆

🔍 正在查詢 法國電力 2040 5.6 (USF2893TAE67) ...
✅ 法國電力 2040 5.6 (USF2893TAE67) 數據取得成功，共 1404 筆

🔍 正在查詢 英美菸草 2052 4.65 (US05526DBV64) ...
✅ 英美菸草 2052 4.65 (US05526DBV64) 數據取得成功，共 750 筆

🔍 正在查詢 輝瑞 2046 4.125 (US717081ED10) ...
⚠️ 無法取得 輝瑞 2046 4.125 (US717081ED10) 的 tkData，跳過

🔍 正在查詢 輝瑞 2053 5.3 (US716973AG71) ...
✅ 輝瑞 2053 5.3 (US716973AG71) 數據取得成功，共 450 筆

📊 所有數據已儲存至 data/MI_歷史數據_全.xlsx，每支債券單獨存放在不同的 Sheet


多檔數據各自儲存單一文件

In [ ]:
import requests
import pandas as pd
from datetime import datetime

# ISIN 對應的 tkData
isin_to_tkdata = {
    'US02209SBF92': ('高特力 2039 5.95', '1,46441569,1330,333'),
    'US037833BX70': ('蘋果 2046 4.65', '1,31618402,1330,333'),
    'US02209SBE28': ('高特力 2039 5.8', '1,46441575,1330,333'),
    'US716973AH54': ('輝瑞 2053 5.3', '1,127132136,1330,333'),
    'US842434DA71': ('南加州天然氣 2054 5.6', ''),  # 尚未查到
    'US872898AJ06': ('台積電 2052 4.5', '1,118393079,16,333'),
    'USF2893TAE67': ('法國電力 2040 5.6', '1,10955366,1330,333'),
    'US05526DBV64': ('英美菸草 2052 4.65', '1,117582253,1330,333'),
    'US717081ED10': ('輝瑞 2046 4.125', ''),  # 尚未查到
    'US716973AG71': ('輝瑞 2053 5.3', '1,127131476,1330,333')
}

# 查詢時間範圍
start_date = "19700201"
end_date = "20250220"

# 遍歷每個 ISIN，查詢價格數據
for isin, (bond_name, tk_data) in isin_to_tkdata.items():
    print(f"\n🔍 正在查詢 {bond_name} ({isin}) ...")

    if not tk_data:
        print(f"⚠️ 無法取得 {bond_name} ({isin}) 的 tkData，跳過")
        # 若無 tkData，直接跳過
        continue

    # 設定 API URL
    url = f"https://markets.businessinsider.com/Ajax/Chart_GetChartData?instrumentType=Bond&tkData={tk_data}&from={start_date}&to={end_date}"
    
    # 發送請求
    response = requests.get(url)

    if response.status_code == 200:
        try:
            data = response.json()
            # 若 API 回傳空數據，則跳過
            if not data:
                print(f"⚠️ {bond_name} ({isin}) 無數據，跳過")
                continue

            # 轉換為 DataFrame
            df = pd.DataFrame(data)
            # 日期格式 yyyy/mm/dd
            df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y/%m/%d')

            # 只保留 Close 欄位
            df = df[['Date', 'Close']]

            # 加入 ISIN 和 Bond Name
            df.insert(1, 'ISIN', isin)
            df.insert(2, 'Bond Name', bond_name)

            # 設定儲存的 Excel 檔名
            excel_filename = f"data/MI_歷史數據_{isin}_{start_date}-{end_date}.xlsx"

            # 儲存為獨立的 Excel 文件
            df.to_excel(excel_filename, index=False)
            print(
                f"✅ {bond_name} ({isin}) "
                f"數據儲存至 {excel_filename}"
            )

        except Exception as e:
            print(f"❌ 解析 {bond_name} ({isin}) JSON 失敗: {e}")
    else:
        print(
            f"❌ {bond_name} ({isin}) 查詢失敗，"
            f"HTTP 狀態碼: {response.status_code}"
        )

print("\n📊 所有數據下載完成！")



🔍 正在查詢 高特力 2039 5.95 (US02209SBF92) ...
✅ 高特力 2039 5.95 (US02209SBF92) 數據儲存至 data/MI_歷史數據_US02209SBF92_19700201-20250220.xlsx

🔍 正在查詢 蘋果 2046 4.65 (US037833BX70) ...
✅ 蘋果 2046 4.65 (US037833BX70) 數據儲存至 data/MI_歷史數據_US037833BX70_19700201-20250220.xlsx

🔍 正在查詢 高特力 2039 5.8 (US02209SBE28) ...
✅ 高特力 2039 5.8 (US02209SBE28) 數據儲存至 data/MI_歷史數據_US02209SBE28_19700201-20250220.xlsx

🔍 正在查詢 輝瑞 2053 5.3 (US716973AH54) ...
✅ 輝瑞 2053 5.3 (US716973AH54) 數據儲存至 data/MI_歷史數據_US716973AH54_19700201-20250220.xlsx

🔍 正在查詢 南加州天然氣 2054 5.6 (US842434DA71) ...
⚠️ 無法取得 南加州天然氣 2054 5.6 (US842434DA71) 的 tkData，跳過

🔍 正在查詢 台積電 2052 4.5 (US872898AJ06) ...
✅ 台積電 2052 4.5 (US872898AJ06) 數據儲存至 data/MI_歷史數據_US872898AJ06_19700201-20250220.xlsx

🔍 正在查詢 法國電力 2040 5.6 (USF2893TAE67) ...
✅ 法國電力 2040 5.6 (USF2893TAE67) 數據儲存至 data/MI_歷史數據_USF2893TAE67_19700201-20250220.xlsx

🔍 正在查詢 英美菸草 2052 4.65 (US05526DBV64) ...
✅ 英美菸草 2052 4.65 (US05526DBV64) 數據儲存至 data/MI_歷史數據_US05526DBV64_19700201-20250220.xlsx

🔍 正在查詢 輝瑞 2046 4.125 (US